In [1]:
from zipfile import ZipFile
import os
import pandas as pd
from otter import Otter
from astropy.coordinates import SkyCoord
import astropy.units as u

In [2]:
zipfile = os.path.join(os.path.dirname(os.getcwd()), 'data', 'test-zip.zip')
datadir = os.path.join(os.path.dirname(os.getcwd()), 'data')
datapath = os.path.join(datadir, os.path.basename(zipfile).replace('.zip', ''))

with ZipFile(zipfile) as z:
    z.extractall(datadir)

In [3]:
# read in the meta file
metapath = os.path.join(datapath, 'meta.csv')
print(metapath)
if not os.path.exists(metapath):
    raise ValueError('meta.csv not found, can not upload this data!')
df = pd.read_csv(metapath)

/home/nfranz/otter/data/test-zip/meta.csv


In [4]:
db = Otter(username='admin@otter', password='insecure')

In [5]:
db.upload(zipfile)

Adding this as a new object...


/home/nfranz/otter/py/otter/otter.py:446: FutureWarning:

In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.



In [20]:
1*u.Unit('mag(AB)').to(u.erg/u.cm**2/u.s/u.Hz)

1.4454397707459234e-20

In [22]:
(1*u.Unit('mag(AB)')).unit

Unit("mag(AB)")